In [2]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

## Load the training data into feature matrix, class labels, and event ids:

In [3]:
from proj1_helpers import *
DATA_TRAIN_PATH = '../Data/train.csv' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

## Training:

In [13]:
from ml_functions import *

#First we build a polynomial model to represent the features
#phi = build_poly(tX, 2)

gamma = 0.1
max_iters = 30
w_initial = np.full((tX.shape[1]), 0.0)
'''w_initial, gamma,  max_iters'''
w = least_squares(y, tX)
w = logistic_regression(y,tX,gamma,max_iters)

/home/zhi/ML2016/ml-project1/ML_methods/ml_functions.py:128: RuntimeWarning: overflow encountered in exp
  if x >= 0:


In [57]:
w
weights = w

## Generate predictions and save ouput in csv format for submission:

In [6]:
DATA_TEST_PATH = '../Data/test.csv' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [7]:
OUTPUT_PATH = '../Data/result.csv' 
# TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test)
print(y_pred.shape)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)

(568238,)
